In [1]:
import pandas as pd
import numpy as np

In [4]:
df=pd.read_csv('pima-indians-diabetes.csv')
df.head()

,pregnancies,glucose,bloodpressure,skinthickness,insulin,bmi,diabetespedigreefunction,age,outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [5]:
df.isnull().sum()

pregnancies                 0
glucose                     0
bloodpressure               0
skinthickness               0
insulin                     0
bmi                         0
diabetespedigreefunction    0
age                         0
outcome                     0
dtype: int64

In [6]:
df.describe()

,pregnancies,glucose,bloodpressure,skinthickness,insulin,bmi,diabetespedigreefunction,age,outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [8]:
df.outcome.value_counts()

outcome
0    500
1    268
Name: count, dtype: int64

In [9]:
268/500

0.536

There is slight **imbalance** in our dataset but since it is not major we will not worry about it!

In [10]:
x=df.drop('outcome',axis='columns')
y=df.outcome

In [13]:
x.shape

(768, 8)

In [14]:
from sklearn.preprocessing import StandardScaler

In [16]:
scaler=StandardScaler()
x_scaled=scaler.fit_transform(x)
x_scaled


array([[ 0.63994726,  0.84832379,  0.14964075, ...,  0.20401277,
         0.46849198,  1.4259954 ],
       [-0.84488505, -1.12339636, -0.16054575, ..., -0.68442195,
        -0.36506078, -0.19067191],
       [ 1.23388019,  1.94372388, -0.26394125, ..., -1.10325546,
         0.60439732, -0.10558415],
       ...,
       [ 0.3429808 ,  0.00330087,  0.14964075, ..., -0.73518964,
        -0.68519336, -0.27575966],
       [-0.84488505,  0.1597866 , -0.47073225, ..., -0.24020459,
        -0.37110101,  1.17073215],
       [-0.84488505, -0.8730192 ,  0.04624525, ..., -0.20212881,
        -0.47378505, -0.87137393]])

In [17]:
from sklearn.model_selection import train_test_split

In [18]:
x_train,x_test,y_train,y_test=train_test_split(x_scaled,y,stratify=y,random_state=10)

In [19]:
x_train.shape

(576, 8)

In [20]:
x_test.shape

(192, 8)

In [22]:
y_train.value_counts()

outcome
0    375
1    201
Name: count, dtype: int64

In [23]:
201/375

0.536

In [24]:
y_test.value_counts()

outcome
0    125
1     67
Name: count, dtype: int64

In [26]:
67/125

0.536

In [29]:
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier


In [30]:
scores=cross_val_score(DecisionTreeClassifier(),x,y,cv=5)
scores

array([0.69480519, 0.66233766, 0.72727273, 0.78431373, 0.73856209])

In [31]:
scores.mean()

0.7214582802818097

## Train using Bagging

In [32]:
from sklearn.ensemble import BaggingClassifier

In [34]:
bag_model=BaggingClassifier(
    estimator=DecisionTreeClassifier(),
    n_estimators=50,
max_samples=0.8,
oob_score=True,
random_state=0
)
bag_model.fit(x_train,y_train)
bag_model.oob_score_

0.7447916666666666

In [35]:
bag_model.score(x_test,y_test)

0.7916666666666666

In [37]:
bag_model = BaggingClassifier(
    estimator=DecisionTreeClassifier(), 
    n_estimators=100, 
    max_samples=0.8, 
    oob_score=True,
    random_state=0
)
scores=cross_val_score(bag_model,x,y,cv=5)
scores

array([0.75324675, 0.72727273, 0.74675325, 0.82352941, 0.73856209])

In [38]:
scores.mean()

0.7578728461081402

We can see some improvement in test score with **bagging classifier** as compared to a **standalone classifier**

In [39]:
from sklearn.ensemble import RandomForestClassifier

In [40]:
scores=cross_val_score(RandomForestClassifier(n_estimators=50),x,y,cv=5)
scores

array([0.75324675, 0.71428571, 0.75974026, 0.82352941, 0.73202614])

In [42]:
scores.mean()

0.7565656565656566